</br>
<center><h1 style='color:#276FF5'> AskTom 2025-08-27 </h1></center>

<center><h1 style='color:#1a2f40'> Copy Point Cloud Data from ADB to LAS file and then from LAS file to 3D Tiles </h1></center>

**<span style='color:#1a2f40'>Author:**</span>Renée Wikestad
<br>Product Manager Oracle Spatial & Graph</br>
<br>
**<span style='color:#1a2f40'>Description:**</span> In this notebook we will explore how we can read point cloud data from ADB and save it to a LAS file 
<br> and then to 3D tiles for visualization in Spatial Studio.

In [1]:
import oci
import os
import json
import pandas as pd
import numpy as np
import laspy
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS
import oracledb
from sqlalchemy import create_engine
from sqlalchemy import text
import warnings


print("ok")

ok


<h3 style='color:#1a2f40'>Connect to Oracle Autonomus Database (ADB)</h3>

In [2]:
with open('credentials.json') as f:
    data = json.load(f)
    username = data['username']
    pw = data['password']
    wpw= data['walletpw'] 
    mydsn= data['dnsname']
    confdir= data['tnsname']
    walletdir= data['wallet']
    comid= data['lagring'] 

In [3]:
# Connect to the database
myengine = create_engine(
    f'oracle+oracledb://:@',
    connect_args={
        "user": username,
        "password": pw,
        "dsn": mydns,
        "config_dir": confdir, # directory containing tnsnames.ora
        "wallet_location": walletdir, # directory containing ewallet.pem
        "wallet_password": wpw,  # password for the PEM file
    })

<h3 style='color:#1a2f40'>Read Data from ADB</h3>

In [5]:
# sql guery 
sql_query="select x, y, z, intensity,nrp,nr,classification,scan_angle,color_r,color_g,color_b,treeid,treesp,osx,osy,osz,sx,sy,sz from N1_CULS_PC where treeid in (1101,1102,1103,2102,2110,3103,3113)"

# Create Pandas DataFrame directly from SQL
df = pd.read_sql(sql_query, myengine)

print(df.head())


            x            y        z  intensity  nrp  nr  classification  \
0  480610.612  5601254.139  355.450      30965    1   1               5   
1  480610.624  5601254.202  355.494       8585    2   1               5   
2  480610.636  5601254.266  355.467      25198    2   1               5   
3  480610.646  5601254.326  355.446      22871    2   1               5   
4  480610.660  5601254.391  355.498       1933    2   1               5   

   scan_angle  color_r  color_g  color_b  treeid  treesp     osx      osy  \
0           1        0    43520        0    3103       2  480609  5601255   
1           1        0    43520        0    3103       2  480609  5601255   
2           1        0    43520        0    3103       2  480609  5601255   
3           1        0    43520        0    3103       2  480609  5601255   
4           1        0    43520        0    3103       2  480609  5601255   

   osz     sx     sy     sz  
0    0  0.001  0.001  0.001  
1    0  0.001  0.001  0.00

<h3 style='color:#1a2f40'>Create GeoDataFrame</h3> 

In [6]:
df['geometry'] = df.apply(lambda row: Point(row['x'], row['y'], row['z']), axis=1)
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.set_crs(epsg=25833, inplace=True)

print("GeoDataFrame with CRS:")
print(gdf.head())
print("CRS:", gdf.crs)


GeoDataFrame with CRS:
            x            y        z  intensity  nrp  nr  classification  \
0  480610.612  5601254.139  355.450      30965    1   1               5   
1  480610.624  5601254.202  355.494       8585    2   1               5   
2  480610.636  5601254.266  355.467      25198    2   1               5   
3  480610.646  5601254.326  355.446      22871    2   1               5   
4  480610.660  5601254.391  355.498       1933    2   1               5   

   scan_angle  color_r  color_g  color_b  treeid  treesp     osx      osy  \
0           1        0    43520        0    3103       2  480609  5601255   
1           1        0    43520        0    3103       2  480609  5601255   
2           1        0    43520        0    3103       2  480609  5601255   
3           1        0    43520        0    3103       2  480609  5601255   
4           1        0    43520        0    3103       2  480609  5601255   

   osz     sx     sy     sz                                  ge

<h3 style='color:#1a2f40'>Create LAS file</h3> 

In [8]:

output_dir = "/home/datascience/askTom/data2"
output_file = "CULS_01.las"
output_path = os.path.join(output_dir, output_file)

os.makedirs(output_dir, exist_ok=True)
las.write(output_path)

print(f"LAS file written to: {output_path}")

LAS file written to: /home/datascience/askTom/data2/CULS_01.las


<h3>Py3dtiles is a Python tool and library for manipulating 3D Tiles.</h3>
Used this to convert pointcloud LAS files to a 3D Tiles tileset (tileset.json + pnts files)

In [1]:
!pip show py3dtiles

Name: py3dtiles
Version: 11.0.0
Summary: Python module for 3D tiles format
Home-page: https://py3dtiles.org
Author: 
Author-email: The Py3DTiles team <contact@oslandia.com>
License: Apache-2.0
Location: /opt/conda/lib/python3.11/site-packages
Requires: earcut, lz4, numba, numpy, psutil, pygltflib, pyproj, pyzmq
Required-by: 


In [ ]:
#!py3dtiles convert -help

In [9]:
!py3dtiles convert /home/datascience/askTom/data2/CULS_01.las \
    --out /home/datascience/askTom/data/culs_01_3dtiles \
    --srs_in 25833 \
    --srs_out 4978

Then make a zip file: 
<br> <i>zip -r CULS_01_3DT.zip culs_01_3dtiles/ </i> 
<br> 
<br>Upload to Spatial Studio with dataset in Cesium format (tileset)